In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# -*- coding: utf-8 -*-
import argparse
import math
import sys
import time
import copy
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization#, regularizers
from keras.layers.noise import GaussianNoise
from keras.layers import Conv1D, MaxPooling1D
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils.np_utils import to_categorical
#K.set_image_dim_ordering('th')
#print(K.image_data_format())

## required for efficient GPU use
#import tensorflow as tf
#from keras.backend import tensorflow_backend
#config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
#session = tf.Session(config=config)
#tensorflow_backend.set_session(session)
## required for efficient GPU use

import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
model_path='/content/drive/MyDrive/Colab Notebooks/CSCI 356 Term Project/Shallow CNN Model/results/cnn_shallow_train__test+.h5'

# prepare callbacks
callbacks = [
    EarlyStopping(
        monitor='val_acc', 
        patience=10,
        mode='max',
        verbose=1),
    ModelCheckpoint(model_path,
        monitor='val_acc', 
        save_best_only=True, 
        mode='max',
        verbose=0)
]

In [4]:
# get data

class dataset:
    kdd_train__2labels = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/CSCI 356 Term Project/KDD Dataset Preprocessing/kdd_train__2labels.pkl") #train-
    kdd_test_2labels = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/CSCI 356 Term Project/KDD Dataset Preprocessing/kdd_test_2labels.pkl") #test+
    
from sklearn import model_selection as ms
from sklearn import preprocessing as pp

class preprocess:
    
    output_columns_2labels = ['is_Normal','is_Attack']
    
    x_input = dataset.kdd_train__2labels.drop(output_columns_2labels, axis = 1)
    y_output = dataset.kdd_train__2labels.loc[:,output_columns_2labels]

    x_test_input = dataset.kdd_test_2labels.drop(output_columns_2labels, axis = 1)
    y_test = dataset.kdd_test_2labels.loc[:,output_columns_2labels]

    ss = pp.StandardScaler()

    x_train_ = ss.fit_transform(x_input)
    x_test = ss.fit_transform(x_test_input)

    y_train_ = y_output.values
    y_test = y_test.values
    
    print('x_train shape: {}'.format(x_train_.shape))
    print('y_train shape: {}'.format(y_train_.shape))
    print('x_test_ shape: {}'.format(x_test.shape))
    print('y_test_ shape: {}'.format(y_test.shape))

x_train shape: (25192, 122)
y_train shape: (25192, 2)
x_test_ shape: (22544, 122)
y_test_ shape: (22544, 2)


In [5]:
#reshape data
X_train_ = np.reshape(preprocess.x_train_, (preprocess.x_train_.shape[0], preprocess.x_train_.shape[1], 1))
X_test = np.reshape(preprocess.x_test, (preprocess.x_test.shape[0], preprocess.x_test.shape[1], 1))

print('train shape after reshape: {}'.format(X_train_.shape))
print('test shape after reshape: {}'.format(X_test.shape))

train shape after reshape: (25192, 122, 1)
test shape after reshape: (22544, 122, 1)


In [6]:
#size of parameters
batch_size = 32
num_classes = 2
epochs = 4
filter_size= 3
droprate=0.50

In [7]:
#Start Neural Network
model = Sequential()

#convolution 1st layer
model.add(Conv1D(64, kernel_size=(filter_size), padding="same",
                 activation='relu',
                 input_shape=(122, 1)))
model.add(BatchNormalization())
model.add(Dropout(droprate))
model.add(MaxPooling1D(strides=2))

#FCN layer
model.add(Flatten())
model.add(Dense(64,use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(droprate))
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss="binary_crossentropy", optimizer="Adam", metrics=['accuracy'])

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 122, 64)           256       
                                                                 
 batch_normalization (BatchN  (None, 122, 64)          256       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 122, 64)           0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 61, 64)           0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 3904)              0         
                                                  

In [8]:
#Save Model=ON
history = model.fit(X_train_, preprocess.y_train_,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, preprocess.y_test),shuffle=True,callbacks=callbacks)

score = model.evaluate(X_test, preprocess.y_test,verbose=0)

#print loss and accuracy
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 25192 samples, validate on 22544 samples
Epoch 1/4
25152/25192 [============================>.] - ETA: 0s - loss: 0.1188 - acc: 0.9649

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


25192/25192 [==============================] - 18s 699us/sample - loss: 0.1186 - acc: 0.9649 - val_loss: 0.6815 - val_acc: 0.7747
Epoch 2/4
25192/25192 [==============================] - 17s 663us/sample - loss: 0.0722 - acc: 0.9743 - val_loss: 0.6809 - val_acc: 0.8062
Epoch 3/4
25192/25192 [==============================] - 17s 660us/sample - loss: 0.0600 - acc: 0.9796 - val_loss: 0.7360 - val_acc: 0.8153
Epoch 4/4
25192/25192 [==============================] - 17s 657us/sample - loss: 0.0558 - acc: 0.9796 - val_loss: 0.7731 - val_acc: 0.8127
Test loss: 0.7731061974808712
Test accuracy: 0.8127218


In [10]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(X_test)
print(y_pred)
y_pred = np.argmax(y_pred, axis=1)
print(y_pred)

predict_proba = model.predict(X_test)
predict_classes = np.argmax(predict_proba, axis=1)

target_names = ['Normal', 'Attack']
print(classification_report(np.argmax(preprocess.y_test, axis=1), y_pred, target_names=target_names, digits=4))
print(confusion_matrix(np.argmax(preprocess.y_test, axis=1), y_pred))

[[2.6324357e-10 1.0000000e+00]
 [1.9380335e-09 1.0000000e+00]
 [9.9958211e-01 4.1787027e-04]
 ...
 [8.5064575e-02 9.1493541e-01]
 [1.0000000e+00 2.9691767e-09]
 [1.9604065e-06 9.9999809e-01]]
[1 1 0 ... 1 0 1]
              precision    recall  f1-score   support

      Normal     0.7221    0.9190    0.8087      9711
      Attack     0.9227    0.7323    0.8166     12833

    accuracy                         0.8127     22544
   macro avg     0.8224    0.8256    0.8126     22544
weighted avg     0.8363    0.8127    0.8132     22544

[[8924  787]
 [3435 9398]]
